Lecture: AI I - Basics 

Previous:
[**Chapter 4.2: Machine Learning with scikit-learn**](../04_ml/02_machine_learning.ipynb)

---

# Chapter 4.3: Evaluation with scikit-learn

- [](#)

---

Lecture: AI I - Basics 

Exercise: [**Exercise 4.3: Evaluation with scikit-learn**](../04_ml/exercises/03_evaluation.ipynb)

Next: [**Chapter 5.1: Assesment 1**]()